# Title
[]()

In [1]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re
from itertools import product
import openai

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from file_functions import *
from summary_chain import *

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
def openai_models(env="api_openai", query='gpt'):
    """
    List the availabel OpenAI models.
    Parameters:
        - env (str): Name of environmental variable storing the OpenAI API key.
        - query (str): Search term for filtering models.
    """
    openai.api_key = os.getenv(env)
    response = openai.Model.list()
    filtered_models = [model for model in response['data'] if model['id'].find(query) != -1]

    for item in filtered_models:
        print(item['id'])
    return filtered_models

openai_models()

gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-0301
gpt-3.5-turbo
gpt-3.5-turbo-0613
gpt-3.5-turbo-16k


[<Model model id=gpt-3.5-turbo-16k-0613 at 0x14be6e931d0> JSON: {
   "created": 1685474247,
   "id": "gpt-3.5-turbo-16k-0613",
   "object": "model",
   "owned_by": "openai",
   "parent": null,
   "permission": [
     {
       "allow_create_engine": false,
       "allow_fine_tuning": false,
       "allow_logprobs": true,
       "allow_sampling": true,
       "allow_search_indices": false,
       "allow_view": true,
       "created": 1686675077,
       "group": null,
       "id": "modelperm-RizYeOCRqav9e70KnCJXhMlS",
       "is_blocking": false,
       "object": "model_permission",
       "organization": "*"
     }
   ],
   "root": "gpt-3.5-turbo-16k-0613"
 },
 <Model model id=gpt-3.5-turbo-0301 at 0x14be6e93590> JSON: {
   "created": 1677649963,
   "id": "gpt-3.5-turbo-0301",
   "object": "model",
   "owned_by": "openai",
   "parent": null,
   "permission": [
     {
       "allow_create_engine": false,
       "allow_fine_tuning": false,
       "allow_logprobs": true,
       "allow_sampl

# Set up

In [4]:
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()
qna_dict = dict()

In [22]:
# Create text dictionary
folder_path = '../text/2023-06-14 full' # ** UPDATE REQUIRED**

encoding='ISO-8859-1'
subset=None

text_dict = create_text_dict_from_folder(folder_path, encoding=encoding, subset=subset)


Keys for text_dict: dict_keys([1, 2])



In [33]:
print(text_dict[2])

Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women

Chronic pain is a pervasive health problem and is associated with tremendous socioeconomic costs. However, current pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild hypohydration was shown to increase experimental pain sensitivity in men, but whether this also occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17?-estradiol and progesterone) throughout the menstrual cycle may influence a womans pain sensitivity, as well as hydration levels, suggesting possible interactions between hypohydration and menstrual phase on pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal (ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse the negative effects of hypoh

In [26]:
from textwrap import wrap
wrap(text_dict[2], width=20)

['Hypohydration but', 'not Menstrual Phase', 'Influences Pain', 'Perception in', 'Healthy Women', 'Chronic pain is a', 'pervasive health', 'problem and is', 'associated with', 'tremendous', 'socioeconomic costs.', 'However, current', 'pain treatments are', 'often ineffective', 'due, in part, to the', 'multifactorial', 'nature of pain. Mild', 'hypohydration was', 'shown to increase', 'experimental pain', 'sensitivity in men,', 'but whether this', 'also occurs in women', 'has not been', 'examined.', 'Fluctuations in', 'ovarian hormones', '(i.e., 17?-estradiol', 'and progesterone)', 'throughout the', 'menstrual cycle may', 'influence a woman\x92s', 'pain sensitivity, as', 'well as hydration', 'levels, suggesting', 'possible', 'interactions between', 'hypohydration and', 'menstrual phase on', 'pain. We', 'investigated the', 'effects of mild', 'hypohydration (HYPO,', '24 h of fluid', 'restriction) on', 'ischemic pain', 'sensitivity in 14', 'eumenorrheic women', 'during the early', 'follicul

In [32]:
from textwrap import fill
print(fill(text_dict[2], width=100))

Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women  Chronic pain is a
pervasive health problem and is associated with tremendous socioeconomic costs. However, current
pain treatments are often ineffective due, in part, to the multifactorial nature of pain. Mild
hypohydration was shown to increase experimental pain sensitivity in men, but whether this also
occurs in women has not been examined. Fluctuations in ovarian hormones (i.e., 17?-estradiol and
progesterone) throughout the menstrual cycle may influence a womans pain sensitivity, as well as
hydration levels, suggesting possible interactions between hypohydration and menstrual phase on
pain. We investigated the effects of mild hypohydration (HYPO, 24 h of fluid restriction) on
ischemic pain sensitivity in 14 eumenorrheic women during the early follicular (EF) and mid-luteal
(ML) phases of their menstrual cycle. We also examined whether acute water ingestion could reverse
the negative effects of hypoh

# Prompts 1

In [38]:
prep_step = [
    "Think about why this might be relevant for the audience in the grand scheme of things.\
    \nIdentify 1 or 2 key concepts from this article that would make interesting or helpful health content. \
    Exclude details that do not add value to the audience.\
    \nBased on the key concepts from the previous steps, extract the key points and statistics to",
]

summarize_task = [
    "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\nIf applicable, include a brief description of the research participants, \
    such as age and sex, within the body of the summary. \
    Otherwise, you can skip this step.\
    \nEvaluate whether or not your writing may be confusing or redundant. \
    \nIf so, re-write it so it is clear and concise. Otherwise, keep it the same. \
    \nCreate a journalistic headline to hook the audience.\
    \nReturn your response in this format:\
    \n<headline>\n\n<summary>\
    \nwhere the summary is a publication-ready format.\
    \nDo not label the headline and summary.",
]

system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])

user_simplify_task = [
    """If needed, rewrite the headline and text using terms appropriate for the audience. If not keep it the same.\
    Follow these steps to accomplish this: \
    \n1. Check if the content and language are appropriate for the audience. \
    \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n3. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n4. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience is""",
]

simplify_audience = [
    # "a lay audience",
    "people who are not science experts",
]

user_relevance_task = [
    """Rewrite the headline and text to include a statement of how it is relevant for the audience. \
    Follow these steps to accomplish this: \
    \n1. Think about why this might be relevant for the audience in the grand scheme of things.\
    \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, \
    add a sentence to inform the audience. Otherwise, keep it the same. \
    \n3. Modify the content if needed to reduce redundancy. \
    \n4. Check if the content and language are appropriate for the audience. \
    If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n5. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n6. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience consists of""",
]

relevance_audience = [
    "seniors",
    # "people who enjoy sports",
    "people new to resistance training"
]

## run it

In [7]:
# Set parameters
iteration_id = 1
n_choices = 1
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# # Create simple summaries
# audience = simplify_audience
# simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
#     chatbot_dict[chatbot_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Add relevance
# relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
#     chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Merge the results
# try:
#     qna_dict = merge_all_chaining_results(
#         chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#         empty_columns=empty_columns, chatbot_id=chatbot_id,
#         save_df=save, save_chatbot=save, 
#             csv_path=folder_path,
#     )
#     print(f'\nCompleted merge_all_chaining_results!:)')
# except Exception as error:
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     file = f.f_code.co_filename
#     print(f'An error occurred on line {lineno} in {file}: {error}')
#     print('Unable to merge results')
#     if save:
#         save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
#         print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]


**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 1 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 1 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 144 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\summary_chain.py : This model's maximum context length is 16385 tokens. However, you requested 18619 tokens (9619 in the messages, 9000 in the completion). Please reduce the length of the messages or completion.
	**API request failed for `.summarize()`**
	...Success!
Processing text1_prompt00...
Processing text2_prompt00...


KeyError: 'summary'

In [15]:
from pprint import pprint
pprint(sample_Chaining_attr(chaining_dict, iteration_id)['qna'])

{'article_title': 'ï»¿Effect of dietary sources of calcium and protein on hip '
                  'fractures and falls in older adults in residential care '
                  'cluster randomised controlled trial',
 'date': '2023-06-14 1419',
 'edit task': '\n'
              'If applicable, include a brief description of the research '
              'participants,     such as age and sex, within the body of the '
              'summary.     Otherwise, you can skip this step.    \n'
              'Evaluate whether or not your writing may be confusing or '
              'redundant.     \n'
              'If so, re-write it so it is clear and concise. Otherwise, keep '
              'it the same.     \n'
              'Create a journalistic headline to hook the audience.    \n'
              'Return your response in this format:    \n'
              '<headline>\n'
              '\n'
              '<summary>    \n'
              'where the summary is a publication-ready format.    \n'
     

## Iteration 2 Update scripts after refreshing kernel - original summary

In [16]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, folder_path, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=9000, 
        ):
        self.text = text
        self.folder = re.sub(r'(?:.*\/)?(.*)$', r'\1', folder_path)
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
        print(f'***OpenAI model: {self.model}')

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature, model=None):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        model = self.model if model == None else model
        print(f'\tSending request to {model}')
        print(f'\t\tRequesting {n_choices} choices using {model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        # chatbot = Chaining(self.text, self.folder)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = self.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['date'] = datetime.now().strftime("%Y-%m-%d %H%M")
        self.qna['folder'] = self.folder
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = self.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')

    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance summary': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, folder_path, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, model='gpt-3.5-turbo', max_tokens=1000, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index+1} of {prompts_df.index.max()+1}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(
                    text, folder_path=folder_path, system_role=system_role, 
                    model=model, max_tokens=max_tokens, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                ext=None, json_path=folder_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', model='gpt-3.5-turbo', max_tokens=1000,
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict

# Set parameters
iteration_id = 2
n_choices = 1
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# # Create simple summaries
# audience = simplify_audience
# simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
#     chatbot_dict[chatbot_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Add relevance
# relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
#     chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Merge the results
# try:
#     qna_dict = merge_all_chaining_results(
#         chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#         empty_columns=empty_columns, chatbot_id=chatbot_id,
#         save_df=save, save_chatbot=save, 
#             csv_path=folder_path,
#     )
#     print(f'\nCompleted merge_all_chaining_results!:)')
# except Exception as error:
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     file = f.f_code.co_filename
#     print(f'An error occurred on line {lineno} in {file}: {error}')
#     print('Unable to merge results')
#     if save:
#         save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
#         print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 1 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 1 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Success!
Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (2, 12)
	Original summaries Dataframe columns: Index(['choice', 'date', 'folder', 'article_title', 'system_role', 'model',
       'text', 'prep step', 'summarization task', 'edit task',
       'full summarization task', 'summary'],
      dtype='object')


,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary
0,2023-06-14 1435,2023-06-14 full,ï»¿Effect of dietary sources of calcium and protein on hip fractures and falls in older adults i...,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,ï»¿Effect of dietary sources of calcium and protein on hip fractures and falls in older adults i...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Falls and Fractures in Older Adults\n\nA nutritiona...
1,2023-06-14 1435,2023-06-14 full,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nChronic pai...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Hydration May Influence Pain Perception in Women, Regardless of Menstrual Phase\n\nMild hypohydr..."


In [19]:
save = True
qna_dict = spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=save, filename=None, path=''
    )
qna_dict[iteration_id]

Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (2, 12)
	Original summaries Dataframe columns: Index(['choice', 'date', 'folder', 'article_title', 'system_role', 'model',
       'text', 'prep step', 'summarization task', 'edit task',
       'full summarization task', 'summary'],
      dtype='object')
File saved:  batch_Chaining_summaries_initial2023-06-14_1438.csv
Time completed: 2023-06-14 14:38:46.862615


,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary
0,2023-06-14 1435,2023-06-14 full,ï»¿Effect of dietary sources of calcium and protein on hip fractures and falls in older adults i...,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,ï»¿Effect of dietary sources of calcium and protein on hip fractures and falls in older adults i...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Falls and Fractures in Older Adults\n\nA nutritiona...
1,2023-06-14 1435,2023-06-14 full,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nChronic pai...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Hydration May Influence Pain Perception in Women, Regardless of Menstrual Phase\n\nMild hypohydr..."


## 2.1

In [35]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, folder_path, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=9000, 
        ):
        self.text = text
        self.folder = re.sub(r'(?:.*\/)?(.*)$', r'\1', folder_path)
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
        print(f'***OpenAI model: {self.model}')

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature, model=None):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        model = self.model if model == None else model
        print(f'\tSending request to {model}')
        print(f'\t\tRequesting {n_choices} choices using {model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        # chatbot = Chaining(self.text, self.folder)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = self.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['date'] = datetime.now().strftime("%Y-%m-%d %H%M")
        self.qna['folder'] = self.folder
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = self.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')

    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance summary': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, folder_path, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, model='gpt-3.5-turbo', max_tokens=1000, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index+1} of {prompts_df.index.max()+1}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(
                    text, folder_path=folder_path, system_role=system_role, 
                    model=model, max_tokens=max_tokens, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Success!')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                ext=None, json_path=folder_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    summary_iteration_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', model='gpt-3.5-turbo', max_tokens=1000,
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    summary_iteration_id = summary_iteration_id if summary_iteration_id else iteration_id
    print('summary_iteration_id:', summary_iteration_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict

# Set parameters
iteration_id = 2.1
n_choices = 4
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# # Create simple summaries
# audience = simplify_audience
# simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
#     chatbot_dict[chatbot_id], iteration_id,
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Add relevance
# relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
#     chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
#     n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
#     )

# # Merge the results
# try:
#     qna_dict = merge_all_chaining_results(
#         chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#         empty_columns=empty_columns, chatbot_id=chatbot_id,
#         save_df=save, save_chatbot=save, 
#             csv_path=folder_path,
#     )
#     print(f'\nCompleted merge_all_chaining_results!:)')
# except Exception as error:
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     file = f.f_code.co_filename
#     print(f'An error occurred on line {lineno} in {file}: {error}')
#     print('Unable to merge results')
#     if save:
#         save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
#         print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 4 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Success!
**Text #2 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 4 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Success!
Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (8, 12)
	Original summaries Dataframe columns: Index(['choice', 'date', 'folder', 'article_title', 'system_role', 'model',
       'text', 'prep step', 'summarization task', 'edit task',
       'full summarization task', 'summary'],
      dtype='object')


,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary
0,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...
1,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,2,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture Risk in Older Adults\n\nA two-year cluster...
2,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,3,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Falls and Fractures in Older Adults in Residential ...
3,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,4,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fractures and Falls in Older Adults in Residential ...
4,2023-06-14 1459,2023-06-14 full,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nChronic pai...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Thirsty Women May Experience Increased Pain Sensitivity, Study Finds\n\nMild hypohydration, or m..."
5,2023-06-14 1459,2023-06-14 full,Hypohydration but not Menstrual Phase Influ

In [37]:
save = True
qna_dict = spreadsheet_columns(qna_dict, chatbot_dict, iteration_id, chatbot_id=None,
    save=save, filename=None, path=folder_path
    )
qna_dict[iteration_id]

Processing text1_prompt00...
Processing text2_prompt00...
Original summaries DataFrame shape: (8, 12)
	Original summaries Dataframe columns: Index(['choice', 'date', 'folder', 'article_title', 'system_role', 'model',
       'text', 'prep step', 'summarization task', 'edit task',
       'full summarization task', 'summary'],
      dtype='object')
File saved:  ../text/2023-06-14 full/batch_Chaining_summaries_initial2023-06-14_1500.csv
Time completed: 2023-06-14 15:00:37.970234


,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary
0,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...
1,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,2,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture Risk in Older Adults\n\nA two-year cluster...
2,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,3,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Falls and Fractures in Older Adults in Residential ...
3,2023-06-14 1459,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,4,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fractures and Falls in Older Adults in Residential ...
4,2023-06-14 1459,2023-06-14 full,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nChronic pai...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"If applicable, include a brief description of the research participants, such as age and sex...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Thirsty Women May Experience Increased Pain Sensitivity, Study Finds\n\nMild hypohydration, or m..."
5,2023-06-14 1459,2023-06-14 full,Hypohydration but not Menstrual Phase Influ

### simple and relevance summaries

In [40]:
class Chaining:
    """
    Parameters:
    -----------
    text : str
        Text to feed to GPT for summarization.

    Optional parameters:
    --------------------
    system_role : str
        The role of the ChatGPT system in the conversation. Default is "You are an expert at science communication."
    temperature : float
        Controls the randomness of responses. Lower values result in more predictable responses. Default is 0.7.
    n_choices : int
        Number of ChatGPT responses to generate. Default is 5.
    max_tokens : int
        Token limit for ChatGPT response. Default is 1000.
    model : str
        ChatGPT model to use. Default is "gpt-3.5-turbo".
    """

    def __init__(self, text, folder_path, system_role="You are a helpful assistant.", 
            model="gpt-3.5-turbo", temperature=0.7, max_tokens=9000, 
        ):
        self.text = text
        self.folder = re.sub(r'(?:.*\/)?(.*)$', r'\1', folder_path)
        self.system_role = system_role
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
        print(f'***OpenAI model: {self.model}')

    def create_prompt(self, task, text):
        """
        Creates a prompt for ChatGPT with the given task and text.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        text : str
            The text to include in the ChatGPT prompt.

        Returns:
        --------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        """
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text delimited by triple backticks: ```{text}``` \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('\tDone creating prompt')
        return messages

    def gpt(self, messages, n_choices, temperature, model=None):
        """
        Sends a request to the ChatGPT API with the given messages.

        Parameters:
        -----------
        messages : list
            A list of dictionaries representing the system and user messages in the prompt.
        n_choices : int
            Number of ChatGPT responses to generate.
        temperature : float
            Controls the randomness of responses. Lower values result in more predictable responses.

        Returns:
        --------
        response : dict
            A dictionary representing the ChatGPT response.
        """
        model = self.model if model == None else model
        print(f'\tSending request to {model}')
        print(f'\t\tRequesting {n_choices} choices using {model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=model, messages=messages, 
            temperature=temperature, 
            max_tokens=self.max_tokens,
            n=n_choices
            )
        print('\tDone sending request to GPT-3')
        return response

    def summarize(self, task, prep_step=None, edit_task=None, n_choices=5):
        """
        Generates summaries from the text using ChatGPT.

        Parameters:
        -----------
        task : str
            The task to include in the ChatGPT prompt.
        prep_step : str, optional
            A preparatory step for the task, if applicable.
        edit_task : str, optional
            The final step for the task, if applicable.
        n_choices : int, optional
            Number of ChatGPT responses to generate. Default is 5.

        Returns:
        --------
        qna : dict
            A dictionary representing the summarization task and the generated summaries.
        """
        # chatbot = Chaining(self.text, self.folder)
        full_task = f'{prep_step} {task} {edit_task}'
        prompt = self.create_prompt(full_task, self.text)
        firstline_pattern = r'\s?(\S*)(\n*)(.+)'
        title = re.match(firstline_pattern, self.text)[0]
        self.qna = dict() 
        self.qna['date'] = datetime.now().strftime("%Y-%m-%d %H%M")
        self.qna['folder'] = self.folder
        self.qna['article_title'] = title
        self.qna['system_role'] = self.system_role
        self.qna['model'] = self.model
        self.qna[f'text'] = self.text
        self.qna['prep step'] = prep_step
        self.qna['summarization task'] = task
        self.qna['edit task'] = edit_task
        self.qna['full summarization task'] = full_task
        self.summaries_dict = dict()
        self.article_title = title
        self.response_regex = r'response_(.*)'
        self.simple_summary_dict = dict()
        self.relevance_dict = dict()
        self.n_previous_prompts = dict()

        try:
            response = self.gpt(prompt, n_choices=n_choices, temperature=self.temperature)
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**API request failed for `.summarize()`**')
            return self.qna
        try:
            for index, choice in enumerate(response.choices):
                self.summaries_dict[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            self.qna.setdefault('summary', [])
            self.qna['summary'].extend([value for value in self.summaries_dict.values()])
            # self.summaries_dict['prep_step'] = prep_step
            # self.summaries_dict['task'] = task
            # self.summaries_dict['edit_task'] = edit_task
            # self.summaries_dict['prompt'] = full_task
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print('\t**Error with response parsing**')

    def simplify(self, simplify_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, 
                    pause_per_request=0
                    ):
        simplify_iteration = len(self.simple_summary_dict) + 1 
        self.n_previous_prompts['simply_summary'] = len(self.simple_summary_dict)
        self.simple_summary_dict[simplify_iteration] = dict()
        if simplify_iteration == None:
            simplify_iteration = 1
        full_simplify_task = f'{simplify_task} {audience}'
        print('simplify_iteration: ', simplify_iteration)
        print('Task:', full_simplify_task)
        summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
        print('summaries_keys: \n\t', summaries_keys)
        for key in summaries_keys:
            new_key = re.sub(self.response_regex, rf'simple_summary\1', key)
            print(f'\t\t...Preparing to summarize {key}')
            simplify_prompt = self.create_prompt(full_simplify_task, self.summaries_dict[key])
            try:
                response = self.gpt(simplify_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.simplify()`**')
                return self.qna
            try:
                self.simple_summary_dict[simplify_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.simple_summary_dict[simplify_iteration][key][index] = {
                        'simple summary choice': index+1, 
                        'simplify task': simplify_task,
                        'audience': audience,
                        'full simplify task': f'{simplify_task} {"for" if audience else ""} {audience}',
                        'simple summary': choice["message"]["content"],
                        'original summary': self.summaries_dict[key]
                    }
                    print(f'\t...Summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.simple_summary_dict[simplify_iteration][new_key] = response
                print(f'\t...Error parsing response for summary request')
            if pause_per_request > 0:
                print(f'[.simplify()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.simple_summary_dict
    
    def add_relevance(self, relevance_task, audience, 
                    model="gpt-3.5-turbo", temperature=0.0, n_choices=1, summary_type='original',
                    # relevance_iteration=None, 
                    pause_per_request=0
                    ):
        relevance_iteration = len(self.relevance_dict) + 1 
        self.n_previous_prompts['relevance'] = len(self.relevance_dict)
        self.relevance_dict[relevance_iteration] = dict()
        if relevance_iteration == None:
            relevance_iteration = 1
        full_relevance_task = f'{relevance_task} {audience}'
        print('relevance_iteration: ', relevance_iteration)
        print('Task:', full_relevance_task)
        if summary_type=='original':
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.response_regex, key)]
            summary_regex = self.response_regex
        else:
            self.simple_summary_response_regex = r'simple_summary_(.*)'
            summaries_keys = [key for key in self.summaries_dict.keys() if re.match(self.simple_summary_response_regex, key)]
            summary_regex = self.simple_summary_response_regex
        print('summaries_keys: \n\t', summaries_keys)
        input_summary_dict = self.summaries_dict if summary_type=='original' else self.simple_summary_dict
        for key in summaries_keys:
            new_key = re.sub(summary_regex, rf'relevance_\1', key)
            print(f'\t\t...Preparing to add relevance to {key}')
            relevance_prompt = self.create_prompt(full_relevance_task, input_summary_dict[key])
            try:
                response = self.gpt(relevance_prompt, n_choices=n_choices, temperature=temperature, model=model)
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                print('\t**API request failed for `.add_relevance()`**')
                return self.qna
            try:
                self.relevance_dict[relevance_iteration][key] = dict()
                for index, choice in enumerate(response.choices):
                    self.relevance_dict[relevance_iteration][key][index] = {
                        'relevance choice': index+1, 
                        'relevance task': relevance_task,
                        'audience': audience,
                        'full relevance task': full_relevance_task,
                        'relevance summary': choice["message"]["content"],
                        'preceding summary': input_summary_dict[key]
                    }
                    print(f'\t...Relevance summary given')
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                filename = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", filename, ":", error)
                self.relevance_summary_dict[relevance_iteration][new_key] = response
                print(f'\t...Error parsing response for relevance request')
            if pause_per_request > 0:
                print(f'[.add_relevance()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                time.sleep(pause_per_request)
        return self.relevance_dict
    
def batch_summarize_chain(text_dict, folder_path, prep_step, summarize_task, edit_task, chaining_bot_dict, iteration_id, 
    system_role=None, model='gpt-3.5-turbo', max_tokens=1000, temperature=0.7, pause_per_request=0, n_choices=5,
    save_outputs=False
    ):
    """
    Summarize multiple texts using the same prompts.
    Parameters:
        - text_dict (dict) A dictionary containing the text data to be summarized. 
            The keys of the dictionary are the text IDs and the values are the full texts.
        - prep_step, summarize_task, edit task (list)
        - qna_dict: Dictionary to store the input and outputs.
        - iteration_id (int, float, or string): Unique ID serving as the key for results in the qna_dict

        iteration_id: int, float or string
            A unique identifier for the current iteration.
        temperature: float, optional (default=0.7)
            The level of "creativity" to use when generating summaries. Higher temperatures will result in more diverse summaries, but may also result in lower quality summaries.
        pause_per_request: int or float, optional (default=0)
            The number of seconds to pause between requests to avoid exceeding API rate limits. Defaults to 0, which means no pause.
        save_outputs: bool, optional (default=False)
            Whether to save the outputs of the summarization process to disk.
        filename: str, optional (default=None)
            The name of the file to save the outputs to. If no filename is specified, a default filename will be used.
        csv_path: str, optional 
            The path to the directory where CSV output files will be saved. Defaults to the 'output' folder in the project directory.
        pickle_path: str, optional 
            The path to the directory where pickle output files will be saved. Defaults to the 'pickles' folder in the project directory.

        Returns:
        --------
        chaining_bot_dict: dict
            A dictionary containing the Chaining instances. 
                The keys of the dictionary are the iteration IDs and the values are dictionaries whose
                values are the Chaining instances.

    """
    prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
        columns=['prep_step', 'summarize_task', 'edit_task'])

    chaining_bot_dict[iteration_id] = dict()
    for key in text_dict:
        text = text_dict[key]
        for index in prompts_df.index:
            print(f'**Text #{key} prompt #{index+1} of {prompts_df.index.max()+1}**')
            task = prompts_df.loc[index, 'summarize_task']
            prep_step = prompts_df.loc[index, 'prep_step']
            edit_task = prompts_df.loc[index, 'edit_task']
            try:
                print('Creating Chaining class instance')
                chatbot = Chaining(
                    text, folder_path=folder_path, system_role=system_role, 
                    model=model, max_tokens=max_tokens, temperature=temperature)
                print('Chaining class instance created')
                chatbot.summarize(
                    task=task, prep_step=prep_step, edit_task=edit_task, n_choices=n_choices
                    )
                chaining_bot_dict[iteration_id][f'text{key}_prompt{"{:02d}".format(index)}'] = chatbot
                print('\t...Completed')
                if pause_per_request > 0:
                    print(f'[batch_summarize()] Sleeping {pause_per_request} sec to avoid exceeding API rate limit')
                    time.sleep(pause_per_request) # Account for API rate limit of 3 API requests/limit 
            except Exception as error:
                exc_type, exc_obj, tb = sys.exc_info()
                f = tb.tb_frame
                lineno = tb.tb_lineno
                file = f.f_code.co_filename
                print("An error occurred on line", lineno, "in", file, ":", error)
                print('\t...Error making chatbot request')
                break
    if save_outputs:
        try:
            save_instance_to_dict(
                chaining_bot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_initial',
                ext=None, json_path=folder_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            file = f.f_code.co_filename
            print(f'An error occurred on line {lineno} in {file}: {error}')
            print('[batch_summarize_chain()] Unable to save API response')

    return chaining_bot_dict

def prompt_chaining_dict(simplify_prompts, audience, simple_summaries_dict, chaining_bot_dict, iteration_id,
    chatbot_id=None, n_choices=None, pause_per_request=0,
    prompt_column='simplify', model='gpt-3.5-turbo'
    # simplify_iteration=None
    ):
    """
    Simplify or add context to a summary.
    """
    # chatbot_id = chatbot_id if chatbot_id else iteration_id
    print('chatbot_id:', chatbot_id)
    prompts_df = pd.DataFrame(product(simplify_prompts, audience), columns=[prompt_column, 'audience'])
    if n_choices == None:
        n_choices = 1 if prompt_column == 'simplify' else 5
    print('n_choices:', n_choices)

    simple_summaries_master_list = []
    for text_prompt_key in chaining_bot_dict.keys():
        print(f'**{text_prompt_key}')

        for index in prompts_df.index:
            prompt = prompts_df.loc[index, prompt_column]
            audience = prompts_df.loc[index, 'audience']
            if prompt_column == 'simplify':
                summary_dict = chaining_bot_dict[text_prompt_key].simplify(
                    prompt, audience, model=model,
                    n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            else: 
                summary_dict = chaining_bot_dict[text_prompt_key].add_relevance(
                    prompt, audience, model=model,
                    n_choices=n_choices, pause_per_request=pause_per_request, 
                    )
            simple_summaries_master_list.append(summary_dict)
  
    simple_summaries_dict[iteration_id] = simple_summaries_master_list
    return simple_summaries_dict

# Set parameters
iteration_id = 2.1
n_choices = 4
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
# save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'

# # Create initial summaries
# chaining_dict = batch_summarize_chain(
#     text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
#     system_role=system_role, model=model,
#     n_choices=n_choices, pause_per_request=pause_per_request,
#     iteration_id=iteration_id, save_outputs=save_outputs
#     )
# qna_dict = spreadsheet_columns(
#     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
#     )

# Create simple summaries
simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
    chatbot_dict[chatbot_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
    )

# Add relevance
relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
    chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
    )

# Merge the results
try:
    qna_dict = merge_all_chaining_results(
        chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
        empty_columns=empty_columns, chatbot_id=chatbot_id,
        save_df=save, save_chatbot=save, 
            csv_path=folder_path,
    )
    print(f'\nCompleted merge_all_chaining_results!:)')
except Exception as error:
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    file = f.f_code.co_filename
    print(f'An error occurred on line {lineno} in {file}: {error}')
    print('Unable to merge results')
    if save:
        save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
        print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

chatbot_id: 2.1
n_choices: 1
**text1_prompt00
simplify_iteration:  1
Task: If needed, rewrite the headline and text using terms appropriate for the audience. If not keep it the same.    Follow these steps to accomplish this:     
1. Check if the content and language are appropriate for the audience.     
2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    
3. Return the final version of the headline and text to be shown to the audience.     
It should be in this format:    
<headline>

<summary>    

where the summary is in paragraph form.    
4. Remove the backticks. Do not label the headline and summary. ",
    

Your audience is people who are not science experts
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04']
		...Preparing to summarize response_01
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...S

,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary,M: original summary content rating,N: original summary language rating,O: top summary,P: simple summary choice,Q: audience,R: simplify task,S: full simplify task,T: simple summary,U: simple summary content rating,V: simple summary language rating,W: top simple summary,X: relevance audience 1,Y: add relevance task people new to resistance training,Z: full relevance task people new to resistance training,AA: summary: people new to resistance training,AB: added relevance content rating,AC: added relevance language rating,AD: top added relevance,AE: relevance audience 2,AF: add relevance task seniors,AG: full add relevance task seniors,AH: summary: seniors
0,2023-06-14 1627,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and s...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...,,,,1,people who are not science experts,"If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...","If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...",Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...,,,,people new to resistance training,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...,,,,seniors,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Improving Calcium and Protein Intake Reduces Fracture and Fall Risk in Older Adults: Study\n\nA ...
1,2023-06-14 1627,2023-06-14 full,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,2,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Effect of dietary sources of calcium and protein on hip fractures and falls in older adults in r...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and s...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,Improving Calcium and Protein Intake Reduces Fracture Risk in Older Adults\n\nA two-year cluster...,,,,1,people who are not science experts,"If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...","If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...",Improving Dairy Intake Can Reduce Fracture Risk in Older Adults\n\nA recent study conducted in A...,,,,people new to resistance training,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Improving Calcium and Protein Intake Through Dairy Foods Reduces Fracture Risk in Older Adults D...,,,,seniors

# *End of Page*